<a href="https://colab.research.google.com/github/Mustafaadeel1/Assignament-Q2/blob/main/langchain_Tool_calling_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain google-generativeai langchain_google_genai -qU

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
import math

In [ ]:
from langchain_core.tools import tool
# Add raper function
@tool
def Calculator(expression):
    """
    Evaluate a mathematical expression with Google Docs-style syntax.

    Parameters:
    expression (str): The mathematical expression to evaluate.
                      It should start with '=' (e.g., '=2+2' or '=SUM(1,2,3)').

    Returns:
    float: The result of the evaluation.
    """
    if not expression.startswith('='):
        raise ValueError("Expression must start with '='.")

    # Remove the '=' from the start
    expression = expression[1:].strip()

    # Define available functions like SUM, AVG
    def SUM(*args):
        return sum(args)

    def AVG(*args):
        return sum(args) / len(args) if args else 0

    # Available namespace for eval
    allowed_functions = {
        'SUM': SUM,
        'AVG': AVG,
        **math.__dict__  # Add all math functions (like sin, cos, etc.)
    }

    try:
        # Evaluate the expression in a safe environment
        result = eval(expression, {"__builtins__": None}, allowed_functions)
    except Exception as e:
        return f"Error in evaluation: {e}"

    return result
import requests
@tool
def fetch_latest_news(api_key, category='general', country='us'):
    """
    Fetch the latest news using the NewsAPI.

    Args:
    - api_key (str): Your NewsAPI key.
    - category (str): The category of news (e.g., 'technology', 'sports', etc.).
    - country (str): The country code for the news source (e.g., 'us', 'in').

    Returns:
    - str: Latest news headlines or an error message.
    """
    api_key = "c14272b7748b418c953f7ee6cf82a3c8"
    url = f"https://newsapi.org/v2/top-headlines?category={category}&country={country}&apiKey={api_key}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            articles = data.get('articles', [])
            if articles:
                news = [f"{i+1}. {article['title']}\n   Source: {article['source']['name']}"
                        for i, article in enumerate(articles[:10])]
                return "\n\n".join(news)
            else:
                return "No news articles found."
        else:
            return f"Error: Unable to fetch news (Status Code: {response.status_code})"
    except Exception as e:
        return f"Error: {e}"
        import yfinance as yf

@tool
def fetch_stock_data(ticker):
    """
    Fetch stock data for the given ticker symbol using Alpha Vantage.

    Args:
    - ticker (str): The stock ticker symbol (e.g., 'AAPL', 'GOOGL').

    Returns:
    - str: Stock information or an error message.
    """
    import requests

    api_key = "AqWbGjogZ8vS2XDCQjo3X3A3495ouUGb"
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={ticker}&apikey={api_key}"

    try:
        response = requests.get(url)
        data = response.json()

        if "Global Quote" in data:
            quote = data["Global Quote"]
            return (
                f"Stock: {ticker.upper()}\n"
                f"Current Price: ${quote.get('05. price', 'N/A')}\n"
                f"Previous Close: ${quote.get('08. previous close', 'N/A')}\n"
                f"Day's High: ${quote.get('03. high', 'N/A')}\n"
                f"Day's Low: ${quote.get('04. low', 'N/A')}\n"
                f"Volume: {quote.get('06. volume', 'N/A')}\n"
            )
        else:
            return "Error: No stock data found for the given ticker."
    except Exception as e:
        return f"Error fetching stock data: {e}"



In [ ]:
tools = [Calculator , fetch_latest_news , fetch_stock_data]

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp",api_key= GOOGLE_API_KEY )

In [ ]:
responce = llm.invoke("What is 3 time 3")
responce.content

'3 times 3 is **9**.'

In [ ]:
from langchain.agents import initialize_agent ,AgentType

In [ ]:
agent = initialize_agent(tools,llm,agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION)

In [ ]:
responce = agent.invoke({"input":"What is 2 multiply by 8"})
responce

{'input': 'What is 2 multiply by 8', 'output': '2 multiplied by 8 is 16.'}

In [ ]:
responce = agent.invoke({"input":"What is 2 multiply by 8"})
responce["output"]

'2 multiplied by 8 is 16.'

In [ ]:
responce = agent.invoke({"input":"What is latest news"})
responce["output"]

'Here are the latest news headlines:\n\n1. Disney Buys Fubo, Killing Venu Lawsuit - Front Office Sports\n2. Deadly winter storm leaves hundreds of thousands without power as heavy snow treks east - The Washington Post\n3. Zendaya and Tom Holland are engaged after she debuts diamond ring at Golden Globes: reports - Yahoo Entertainment\n4. Venezuela’s Dissidents Plot Comeback After Months in Hiding, Exile or Jail - The Wall Street Journal\n5. Report: Christian Watson is believed to have suffered torn ACL - NBC Sports\n6. Biden to ban offshore oil, gas drilling in vast areas ahead of Trump term - Reuters\n7. Live coverage: SpaceX to launch first dedicated Starlink mission of 2025 - Spaceflight Now\n8. US top doctor wants cancer warning on alcohol: What does the science say? - Al Jazeera English\n9. Winter Storm Hits D.C., Maryland, Virginia; Schools Closed - The Weather Channel'

In [ ]:
responce = agent.invoke({"input":"What is latest stock coin Bitcoin"})
responce["output"]

"The current price of Bitcoin (BTC) is $45.3400. The previous close was $43.6000, the day's high is $45.5100, and the day's low is $43.8200. The volume is 2,727,840."